In [1]:
import rasterio

In [2]:
import pandas as pd
import geopandas as gpd
import datetime

In [13]:
pavel = pd.read_csv('../data.csv')

### Vereinfachte Geometriedaten der Lankreise

In [14]:
geometry_lk_simple = pd.read_pickle("../geometry_landkreise_simplified/geometry_lk_simplified_0_001.gzip")

### Pavelstabelle mit geometrie Daten anreichern

In [22]:
pd.set_option('display.max_columns', 500)
pavel

,DatenstandTag,Datum,IdLandkreis,Landkreis,LandkreisTyp,IdBundesland,Bundesland,Flaeche,MeldeTag_AnzahlFallNeu,MeldeTag_AnzahlFall,MeldeTag_AnzahlTodesfallNeu,MeldeTag_AnzahlTodesfall,MeldeTag_Vor7Tagen_AnzahlFallNeu,MeldeTag_Vor7Tagen_AnzahlFall,MeldeTag_Vor7Tagen_AnzahlTodesfallNeu,MeldeTag_Vor7Tagen_AnzahlTodesfall,AnzahlFall,AnzahlFallNeu,AnzahlTodesfall,AnzahlTodesfallNeu,AnzahlGenesen,AnzahlGenesenNeu,PublikationsdauerFallNeu_Min,PublikationsdauerFallNeu_Max,PublikationsdauerFallNeu_Schnitt,PublikationsdauerFallNeu_Median,PublikationsdauerFallNeu_StdAbw,PublikationsdauerFallNeu_Fallbasis,DatenstandTag_Max,Einwohner,Dichte,InzidenzFallNeu,MeldeTag_AnzahlFallNeu_Gestern,MeldeTag_AnzahlFall_Gestern,MeldeTag_AnzahlTodesfallNeu_Gestern,MeldeTag_AnzahlTodesfall_Gestern,MeldeTag_Vor7Tagen_AnzahlFallNeu_Vor8Tagen,MeldeTag_Vor7Tagen_AnzahlFall_Vor8Tagen,MeldeTag_Vor7Tagen_AnzahlTodesfallNeu_Vor8Tagen,MeldeTag_Vor7Tagen_AnzahlTodesfall_Vor8Tagen,InzidenzTodesfallNeu,MeldeTag_InzidenzFall,MeldeTag_InzidenzTodesfall,MeldeTag_Fallsterblichkeit_Prozent,MeldeTag_Vor7Tagen_InzidenzFall,MeldeTag_Vor7Tagen_InzidenzTodesfall,MeldeTag_Vor7Tagen_Fallsterblichkeit_Prozent,InzidenzFall,InzidenzTodesfall,Fallsterblichkeit_Prozent,MeldeTag_InzidenzFall_Gestern,MeldeTag_InzidenzTodesfall_Gestern,MeldeTag_Fallsterblichkeit_Prozent_Gestern,MeldeTag_Vor7Tagen_InzidenzFall_Vor8Tagen,MeldeTag_Vor7Tagen_InzidenzTodesfall_Vor8Tagen,MeldeTag_Vor7Tagen_Fallsterblichkeit_Prozent_Vor8Tagen,MeldeTag_AnzahlFallNeu_7TageSumme,MeldeTag_AnzahlTodesfallNeu_7TageSumme,MeldeTag_Vor7Tagen_AnzahlFallNeu_7TageSumme,MeldeTag_Vor7Tagen_AnzahlTodesfallNeu_7TageSumme,AnzahlFallNeu_7TageSumme,AnzahlFallNeu_7TageSumme_Trend,AnzahlFallNeu_7TageSumme_7_Tage_davor,AnzahlTodesfallNeu_7TageSumme,AnzahlTodesfallNeu_7TageSumme_Trend,AnzahlTodesfallNeu_7TageSumme_7_Tage_davor,AnzahlGenesenNeu_7TageSumme,AnzahlGenesenNeu_7TageSumme_Trend,InzidenzFallNeu_7TageSumme,InzidenzFallNeu_7TageSumme_Trend,InzidenzFallNeu_7TageSumme_7_Tage_davor,InzidenzFallNeu_7TageSumme_Trend_Spezial,InzidenzFallNeu_7TageSumme_R,InzidenzFallNeu_Prognose_1_Wochen,InzidenzFallNeu_Prognose_2_Wochen,InzidenzFallNeu_Prognose_4_Wochen,InzidenzFallNeu_Prognose_8_Wochen,InzidenzFallNeu_Tage_bis_50,InzidenzFallNeu_Tage_bis_100,Kontaktrisiko,MeldeTag_AnzahlFallNeu_Gestern_7TageSumme,MeldeTag_AnzahlTodesfallNeu_Gestern_7TageSumme,MeldeTag_Vor7Tagen_AnzahlFallNeu_Vor8Tagen_7TageSumme,MeldeTag_Vor7Tagen_AnzahlTodesfallNeu_Vor8Tagen_7TageSumme,InzidenzTodesfallNeu_7TageSumme,InzidenzTodesfallNeu_7TageSumme_Trend,InzidenzTodesfallNeu_7TageSumme_7_Tage_davor,InzidenzTodesfallNeu_7TageSumme_Trend_Spezial,DatenstandTag_Diff,MeldeTag_InzidenzFallNeu_Gestern_7TageSumme,AnzahlFallNeu_7TageSumme_Dropped,ProzentFallNeu_7TageSumme_Dropped,MeldeTag_Vor7Tagen_InzidenzFallNeu_Vor8Tagen_7TageSumme,MeldeTag_InzidenzFallNeu_Trend,MeldeTag_InzidenzFallNeu_R,MeldeTag_InzidenzFallNeu_Prognose_4_Wochen,PublikationsdauerFallNeu_Min_Neg,Rang,RangChange,RangYesterday,RangChangeStr
0,388,16.3.2021,16076,LK Greiz,LK,16,Thüringen,845.96,0,0,0,0,0,0,0,0,4414,18,122,0,3614,33,0.0,0.0,0.000000,0.0,0.000000,18.0,388,97398,115.133103,18.480872,18,4414,0,122,9,3917,0,121,0.000000,0.0,0.0,NaN,0.0,0.0,NaN,4531.920573,125.259246,2.763933,4531.920573,125.259246,2.763933,4021.643155,124.232530,3.089099,421,0,0,0,497,1.904215,261,1,1.000000,1,195,2.191011,510.277418,1.904215,267.972648,1.887652,1.444900,963.226288,1818.236216,6478.795296,9999.900000,-25.593664,-17.956687,29.662218,478,0,254,0,1.026715,1.000000,1.026715,1.000000,0,490.769831,19,3.822938,260.785642,1.881890,1.435195,6155.375868,0.0,1,0,1,NaN
1,388,16.3.2021,16066,LK Schmalkalden-Meiningen,LK,16,Thüringen,1210.73,0,0,0,0,0,0,0,0,6745,10,250,4,5943,34,0.0,2.0,0.285714,0.0,0.611250,14.0,388,124916,103.174118,8.005380,12,6745,0,250,5,6348,0,233,3.202152,0.0,0.0,NaN,0.0,0.0,NaN,5399.628550,200.134490,3.706449,5399.628550,200.134490,3.706449,5081.814980,186.525345,3.670447,297,1,0,0,397,1.718615,231

In [15]:
merged = gpd.GeoDataFrame(pavel.join(geometry_lk_simple, on='Landkreis'))

### Nur ein Teil der Spalten übernehmen

In [25]:
subset = merged[['Landkreis', 'Bundesland', 'Einwohner', 'AnzahlFallNeu', 'AnzahlFallNeu_7TageSumme_Dropped', 'AnzahlTodesfallNeu_7TageSumme', 'Fallsterblichkeit_Prozent', 'Kontaktrisiko', 'InzidenzFallNeu_7TageSumme', 'PublikationsdauerFallNeu_Schnitt', 'InzidenzFallNeu_7TageSumme_Trend_Spezial', 'geometry']]
subset['Kehrwert_risiko'] = 1/subset['Kontaktrisiko']  # zur vereinfachten Farbzuweisung in der Karte 

/Users/bjoern/Working/covid19/Corona/venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


### Entfernen der Bundesländer/Republik Zeilen

In [26]:
subset = subset[subset.geometry != None]

In [27]:
subset

,Landkreis,Bundesland,Einwohner,AnzahlFallNeu,AnzahlFallNeu_7TageSumme_Dropped,AnzahlTodesfallNeu_7TageSumme,Fallsterblichkeit_Prozent,Kontaktrisiko,InzidenzFallNeu_7TageSumme,PublikationsdauerFallNeu_Schnitt,InzidenzFallNeu_7TageSumme_Trend_Spezial,geometry,Kehrwert_risiko
0,LK Greiz,Thüringen,97398,18,19,1,2.763933,29.662218,510.277418,0.000000,1.887652,"MULTIPOLYGON (((12.19167 50.96358, 12.18251 50...",0.033713
1,LK Schmalkalden-Meiningen,Thüringen,124916,10,20,17,3.706449,52.891782,317.813571,0.285714,1.699696,"MULTIPOLYGON (((10.47199 50.84829, 10.41141 50...",0.018907
2,SK Frankenthal,Rheinland-Pfalz,48762,14,1,6,2.858958,65.693951,170.214511,0.692308,2.555112,"MULTIPOLYGON (((8.40205 49.58243, 8.38594 49.5...",0.015222
3,LK Vogtlandkreis,Sachsen,225997,68,40,17,4.355494,70.855860,324.783072,0.826087,1.241546,"MULTIPOLYGON (((12.33201 50.66915, 12.31892 50...",0.014113
4,SK Pirmasens,Rheinland-Pfalz,40231,7,0,0,5.324814,72.460713,169.023887,0.000000,2.332820,"MULTIPOLYGON (((7.58740 49.23320, 7.57078 49.2...",0.013801
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,LK Rottweil,Baden-Württemberg,139878,-1,0,4,2.723662,2660.671744,30.026166,1.000000,0.357636,"MULTIPOLYGON (((8.59834 48.40543, 8.51094 48.4...",0.000376
425,LK Kaiserslautern,Rheinland-Pfalz,105979,0,0,0,2.195824,2714.769011,14.153747,NaN,0.743580,"MULTIPOLYGON (((7.74158 49.59036, 7.69627 49.5...",0.000368
426,LK Vulkaneifel,Rheinland-Pfalz,60646,1,0,0,3.806934,2912.484249,21.435874,0.000000,0.457643,"MULTIPOLYGON (((6.42349 50.36871, 6.41304 50.3...",0.000343
427,LK Cochem-Zell,Rheinland-Pfalz,61375,0,1,0,3.003195,5558.504578,13.034623,NaN,0.394344,"MULTIPOLYGON (((7.07428 50.27758, 7.06878 50.2...",0.000180


In [28]:
now = datetime.datetime.now()
subset.to_file("../leaflet-website/risk_{:%Y-%m-%d}.geojson".format(now), driver='GeoJSON')